In [1]:
%load_ext lineapy

In [2]:
import subprocess
import pathlib
import inspect
import json
import datetime
print('current UTC:', datetime.datetime.utcnow())

current UTC: 2022-10-13 04:04:09.628469


# Benchmark Model (MonteCarlo Method Implemented in C++)

Here, we are using a Monte Carlo method(implemented in C++) as the benchmark. Here are the steps:

* (optional) Modify the source code if necessary.
* Compile the source code to a shared library.
* Register the shared library in python as a python function.
* Call the new function to get the price from this model.


## Compile Shared Library from C++ Code

Here we are using `subprocess` to compile a shared library from C++ and save the source code as an artifact.

We want to save the source code as an artifact since during the validation process, the C++ code might need to modify to accomodate the model we are validating.

In [3]:
source_code_path = 'lib.cpp'
shared_library_path = './libTest.so'

In [4]:
def get_source_code(source_code_path, shared_library_path):
    p = subprocess.run(['g++', '-fPIC', '-shared', '-o', shared_library_path, source_code_path])
    source_code = pathlib.Path(source_code_path).read_text()
    return source_code 

source_code = get_source_code(source_code_path, shared_library_path)
print(source_code)
lineapy.save(source_code, 'cpp_source_code')

// g++ -fPIC -shared -o libTest.so lib.cpp

#include <iostream>
#include <cmath>
#include <algorithm>    // Needed for the "max" function

int Function(int num) 
{
    std::cout << "Num = " << num << std::endl;
    return num;
}

double gaussian_box_muller() {
    double x = 0.0;
    double y = 0.0;
    double euclid_sq = 0.0;

    // Continue generating two uniform random variables
    // until the square of their "euclidean distance" 
    // is less than unity
    do {
        x = 2.0 * rand() / static_cast<double>(RAND_MAX)-1;
        y = 2.0 * rand() / static_cast<double>(RAND_MAX)-1;
        euclid_sq = x*x + y*y;
    } while (euclid_sq >= 1.0);

    return x*sqrt(-2*log(euclid_sq)/euclid_sq);
}

double monte_carlo_call_price(const int& num_sims, const double& S, const double& K, const double& r, const double& v, const double& T) {
    double S_adjust = S * exp(T*(r-0.5*v*v));
    double S_cur = 0.0;
    double payoff_sum = 0.0;

    for (int i=0; i<num_sims; i++) {
        double

LineaArtifact(name='cpp_source_code', _version=0)

## Register the Shared Library into Python Functions

In [5]:
import ctypes
import sys
import os 

def get_DLL_handle(dllpath):
    handle = ctypes.CDLL(dllpath)     
    handle.My_mc_call.argtypes = [ctypes.c_int, ctypes.c_double, ctypes.c_double, ctypes.c_double, ctypes.c_double, ctypes.c_double] 
    handle.My_mc_call.restype = ctypes.c_double
    handle.My_mc_put.argtypes = [ctypes.c_int, ctypes.c_double, ctypes.c_double, ctypes.c_double, ctypes.c_double, ctypes.c_double] 
    handle.My_mc_put.restype = ctypes.c_double
    return handle

dll_handle= get_DLL_handle(shared_library_path)

def call(num_sims, S, K, r, v, T):
    result = dll_handle.My_mc_call(num_sims, S, K, r, v, T)
    print(result)
    return result

def put(num_sims, S, K, r, v, T):
    result = dll_handle.My_mc_put(num_sims, S, K, r, v, T)
    print(result)
    return result    

### Load option parameters

Save the parameter as an artifact to make sure what exactly parameters are we passing into the pricer

In [6]:
cpp_config_path = '../config/option_config.json'
params = json.load(open(cpp_config_path,'r'))
lineapy.save(params, 'cpp_config')
print(params)

{'option_type': 'call', 'num_sims': 1000, 'S': 100, 'K': 100, 'r': 0.01, 'v': 0.25, 'T': 1}


## Call the Pricer (Monte Carlo Methods) and Cave the Result as an Artifact.

In [7]:
call_param = {k:v for k, v in params.items() if k in inspect.getfullargspec(call).args}
call_price = call(**call_param)

10.98079096253393payoff_sum = 11091.1
r = 0.01
T = 1
result10.9808



In [8]:
lineapy.save(call_price, 'cpp_price')

LineaArtifact(name='cpp_price', _version=0)

In [9]:
art = lineapy.get('cpp_price')
print(art.get_code())

import ctypes
import inspect
import json

shared_library_path = "./libTest.so"


def get_DLL_handle(dllpath):
    handle = ctypes.CDLL(dllpath)
    handle.My_mc_call.argtypes = [
        ctypes.c_int,
        ctypes.c_double,
        ctypes.c_double,
        ctypes.c_double,
        ctypes.c_double,
        ctypes.c_double,
    ]
    handle.My_mc_call.restype = ctypes.c_double
    handle.My_mc_put.argtypes = [
        ctypes.c_int,
        ctypes.c_double,
        ctypes.c_double,
        ctypes.c_double,
        ctypes.c_double,
        ctypes.c_double,
    ]
    handle.My_mc_put.restype = ctypes.c_double
    return handle


dll_handle = get_DLL_handle(shared_library_path)


def call(num_sims, S, K, r, v, T):
    result = dll_handle.My_mc_call(num_sims, S, K, r, v, T)
    print(result)
    return result


cpp_config_path = "../config/option_config.json"
params = json.load(open(cpp_config_path, "r"))
call_param = {k: v for k, v in params.items() if k in inspect.getfullargspec(call).arg